Topic
0. open
1. review
  
   1.1 1, 2일차 내용에 대한 review( naver 영화 한줄평을 이용 )
2. topic

   2.1 정수 인코딩( Integer encoding )

   2.2 원-핫 인코딩( One-hot encoding )
   
   2.3 Byte Pair Encoding, BPE )
   
   2.4 데이터 분리( Splitting Data )
   
3. Q & A
4. close

# 1. Review

# Naver 영화 한줄평 crawling & Test preprocessing


In [1]:
import requests
from bs4 import BeautifulSoup

# 한줄평 crawling

In [2]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321#'

In [3]:
resp = requests.get( url )
html = BeautifulSoup( resp.content, 'lxml' )

In [4]:
score_result = html.find( 'div', { 'class' : 'score_result' } )
lis = score_result.findAll( 'li' )

In [6]:
comments = []
for li in lis:
    review_text = li.find( 'p' ).getText()
    comments.append( review_text.strip() )
print( comments )

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


In [7]:
for comment in comments:
    print( comment + '\n' )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.

충무로: 이거 어케하는거냐?

아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.

촬영감독의 영혼까지 갈아넣은 마스터피스

오스카 작품상 탔어도 할말 없었을것 같다.



### crawling 결과 파일 저장 / 읽기

In [8]:
with open( 'comment_1917.txt', 'w', encoding = 'utf-8' ) as f:
    for comment in comments:
        f.write( comment + '\n' )

In [9]:
movie_comments = []
with open( 'comment_1917.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        movie_comments.append( line.rstrip( '\n' ) )
print( movie_comments )

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


### 파이썬 리스트 내용을 하나의 문자열로 변경

In [10]:
movie_comment = ''
for comment in movie_comments:
    movie_comment += comment 
print( movie_comment )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.충무로: 이거 어케하는거냐?아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.


# Text preprocessing

In [11]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import re

In [12]:
okt = Okt()

## 형태소 추출

In [14]:
morphemes = []
for morphs in okt.morphs( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


## 형태소 추출 및 어간 추출

In [15]:
morphemes = []
for morphs in okt.morphs( movie_comment, stem = True ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미치다', '.', '넷플릭스', '가', '일상', '화', '되다', '시대', '에', '극장', '이', '존재', '하다', '이유', '를', '증명', '해주다', '.', '충무로', ':', '이', '거', '어케', '하다', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받다', '이유', '가', '고스', '란', '히', '녹다', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '하다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣다', '마스터피스', '오스카', '작품', '상', '타다', '하다', '없다', '같다', '.']


## 형태소 추출 및 정규화

In [16]:
morphemes = []
for morphs in okt.morphs( movie_comment, norm = True ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어떻게', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


## 명사 분리

In [17]:
morphemes = []
for morphs in okt.nouns( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '넷플릭스', '일상', '시대', '극장', '존재', '이유', '증명', '충무로', '거', '어케', '아카데미', '촬영상', '음향효과', '효과', '이유', '고스', '란', '영화', '로', '관람', '는걸', '추천', '촬영감독', '영혼', '마스터피스', '오스카', '작품']


## 어절 분리

In [18]:
morphemes = []
for morphs in okt.phrases( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이 영화', '넷플릭스', '일상화', '일상화된 시대', '극장', '존재', '존재해야하는 이유', '증명', '충무로', '이거', '이거 어케', '아카데미', '촬영상', '음향효과상', '시각효과상', '이유', '고스란', '영화', 'IMAX로', 'IMAX로 관람하는걸', 'IMAX로 관람하는걸 추천', '촬영감독', '촬영감독의 영혼', '마스터피스오스카', '마스터피스오스카 작품상', '시대', '어케', '고스', 'IMAX', '관람하', '는걸', '추천', '영혼', '마스터피스', '오스카', '작품상']


## 품사 태깅

In [21]:
morphemes = []
for morphs in okt.pos( movie_comment ): #, join = True ):
    morphemes.append( morphs )
print( morphemes )

[('이', 'Noun'), ('영화', 'Noun'), ('는', 'Josa'), ('미쳤다', 'Adjective'), ('.', 'Punctuation'), ('넷플릭스', 'Noun'), ('가', 'Josa'), ('일상', 'Noun'), ('화', 'Suffix'), ('된', 'Verb'), ('시대', 'Noun'), ('에', 'Josa'), ('극장', 'Noun'), ('이', 'Josa'), ('존재', 'Noun'), ('해야하는', 'Verb'), ('이유', 'Noun'), ('를', 'Josa'), ('증명', 'Noun'), ('해준다', 'Verb'), ('.', 'Punctuation'), ('충무로', 'Noun'), (':', 'Punctuation'), ('이', 'Determiner'), ('거', 'Noun'), ('어케', 'Noun'), ('하는거냐', 'Verb'), ('?', 'Punctuation'), ('아카데미', 'Noun'), ('에서', 'Josa'), ('촬영상', 'Noun'), (',', 'Punctuation'), ('음향효과', 'Noun'), ('상', 'Suffix'), (',', 'Punctuation'), ('시', 'Modifier'), ('각', 'Modifier'), ('효과', 'Noun'), ('상', 'Suffix'), ('을', 'Josa'), ('받은', 'Verb'), ('이유', 'Noun'), ('가', 'Josa'), ('고스', 'Noun'), ('란', 'Noun'), ('히', 'Adverb'), ('녹아있는', 'Verb'), ('영화', 'Noun'), ('.', 'Punctuation'), ('IMAX', 'Alpha'), ('로', 'Noun'), ('관람', 'Noun'), ('하', 'Suffix'), ('는걸', 'Noun'), ('추천', 'Noun'), ('한다', 'Verb'), ('.', 'Punctuation'), ('촬영감독', 'N

## 불용어 파일 Load

In [22]:
stop_words = []
with open( 'stopwords.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append( line.rstrip( '\n' ) )
        
print( stop_words )

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

### 불용어 제거

In [23]:
results = []
for word in okt.morphs( movie_comment ):
    if word not in stop_words:
        results.append( word )
        
print( results )

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '일상', '화', '된', '시대', '극장', '이', '존재', '해야하는', '이유', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '촬영상', ',', '음향효과', '상', ',', '시', '효과', '상', '받은', '이유', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '관람', '는걸', '추천', '한다', '.', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '.']


### cleaning( 특수문제 제거 )

In [24]:
r = '[-=+,#/\?:^$.@*\"~&%.!\'|\(\)\[\]\<\>`\']'

movie_comment = re.sub( r, '', movie_comment )
print( movie_comment )

이 영화는 미쳤다 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다충무로 이거 어케하는거냐아카데미에서 촬영상 음향효과상 시각효과상을 받은 이유가 고스란히 녹아있는 영화 IMAX로 관람하는걸 추천한다촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다


In [25]:
results = []
for word in okt.morphs( movie_comment ):
    if word not in stop_words:
        results.append( word )
        
print( results )

['이', '영화', '는', '미쳤다', '넷플릭스', '일상', '화', '된', '시대', '극장', '이', '존재', '해야하는', '이유', '증명', '해준다', '충무로', '이', '거', '어케', '하는거냐', '아카데미', '촬영상', '음향효과', '상', '시', '효과', '상', '받은', '이유', '고스', '란', '히', '녹아있는', '영화', 'IMAX', '관람', '는걸', '추천', '한다', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것']


### NLTK - word_tokenize() 사용 결과

In [26]:
word_tokens = word_tokenize( movie_comment )
print( word_tokens )

['이', '영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것', '같다']


In [27]:
results = []
for word in word_tokens:
    if word not in stop_words:
        results.append( word )
print( results )

['이', '영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것']


In [28]:
from konlpy.corpus import kolaw
from konlpy.corpus import kobill

In [29]:
kolaw.open( 'constitution.txt' ).read()[ :30 ]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국'

In [30]:
kobill.open( '1809890.txt' ).read()[ :30 ]

'지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n'

## 한나눔( Hannanum ) 형태소 분석기

In [31]:
from konlpy.tag import Hannanum

In [32]:
hannanum = Hannanum()

In [35]:
hannanum.nouns( '나는 자연어 처리를 공부한다.' )

['나', '자연어', '처리', '공부']

In [36]:
hannanum.pos( '나는 자연어 처리를 공부한다.' )

[('나', 'N'),
 ('는', 'J'),
 ('자연어', 'N'),
 ('처리', 'N'),
 ('를', 'J'),
 ('공부', 'N'),
 ('하', 'X'),
 ('ㄴ다', 'E'),
 ('.', 'S')]